<h1><center>Assignment 3</center></h1>

<!-- <center>![University logo](images/logo.png)</center> -->
<div style="text-align:center"><img src="images/logo.png" alt="drawing" width="200"/>

<h2><center>Student Id : 46312625</center></h2>
<h2><center>Proteek Kumar Sanyal</center></h2>


# Content

1. [Data Loading](#1-data-loading)

    1.1. [Loading NYC Data](#12-loading-nyc-data)

    1.2. [Comparing & discussing 3 months](#12-analysis-of-3-months-data)

2. [Data Exploration and Pre-processing](#2-data-exploration-and-pre-processing)
    
    2.1. [Missing Values](#21-missing-values)

    2.2. [Noisy Values](#22-noisyerroneous-values)
    
    2.3. [Correlation](#23-correlation)

3. [Featurization](#3-featurization)

    3.1. [Rush hour](#31-rush-hour)

    3.2. [Complexity of the trip](#32-complexity-of-the-trip)

    3.3. [Pickup/Dropout frequency](#33-pickup-and-dropoff-frequency)

4. [Data Analysis](#4-data-analysis)

    4.1. [Vendors Ranking](#41-rank-the-vendors-by-popularity)

    4.2. [Peak travel hours](#42-peak-travel-hours)

    4.3. [Average distance of trips](#43-average-distance-of-the-trips-on-weekdays-and-weekends)

    4.4. [Average number of passengers in a trip](#44-average-number-of-passengers-in-a-trip-on-weekdays-and-weekends)
    
    4.5. [Correlation b/w trip and tip](#45-correlation-between-the-fare-about-and-the-tip)

    4.6. [Correlation b/w fare and no. of passengers](#46-correlation-between-the-fare-amount-and-the-number-of-passengers)

5. [Conclusion](#5-conclusion)

    5.1. [Key finding](#51-key-findings)

    5.2. [Challenges](#52-challenges)

    5.3. [Additional Analysis](#53-addition-analysis)

# 1. Data Loading

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import main as m
import matplotlib.pyplot as plt

## 1.2 Loading NYC data

In [ ]:
jan_df = m.get_january_dataframe()
mar_df = m.get_march_dataframe()
june_df = m.get_june_dataframe()

In [ ]:
# Januray dataframe
jan_df.head(10)

In [ ]:
# March dataframe
mar_df.head(10)

In [ ]:
# June dataframe
june_df.head(10)

## 1.2 Analysis of 3 months data

### Trip distance

In [ ]:
# plotting trip distance all three months together for only 2023 year
jan_df_2023 = jan_df[jan_df['tpep_pickup_datetime'].dt.year == 2023]
mar_df_2023 = mar_df[mar_df['tpep_pickup_datetime'].dt.year == 2023]
june_df_2023 = june_df[june_df['tpep_pickup_datetime'].dt.year == 2023]

sns.set()
plt.plot(jan_df_2023['tpep_pickup_datetime'], jan_df_2023['trip_distance'], 'o', color='red')
plt.plot(mar_df_2023['tpep_pickup_datetime'], mar_df_2023['trip_distance'], 'o', color='blue')
plt.plot(june_df_2023['tpep_pickup_datetime'], june_df_2023['trip_distance'], 'o', color='green')
plt.xlabel('pickup_datetime')
plt.ylabel('trip_distance')
plt.title('Trip distance vs pickup_datetime for 2023')
plt.savefig('image/trip_distance.png')
plt.show()


### Payment Type

In [ ]:
# plotting payment type for all three months together
sns.set()
plt.plot(jan_df_2023['tpep_pickup_datetime'], jan_df_2023['payment_type'], 'o', color='red')
plt.plot(mar_df_2023['tpep_pickup_datetime'], mar_df_2023['payment_type'], 'o', color='blue')
plt.plot(june_df_2023['tpep_pickup_datetime'], june_df_2023['payment_type'], 'o', color='green')
plt.xlabel('pickup_datetime')
plt.ylabel('payment_type')
plt.title('Payment type vs pickup_datetime for 2023')
plt.savefig('image/payment_type.png')
plt.show()

# plotting the histogram of payment type frequency for all three months together
sns.set()
plt.hist(jan_df_2023['payment_type'], bins=20, color='red')
plt.hist(mar_df_2023['payment_type'], bins=20, color='blue')
plt.hist(june_df_2023['payment_type'], bins=20, color='green')
plt.xlabel('payment_type')
plt.ylabel('frequency')
plt.title('Payment type frequency for 2023')
plt.savefig('image/payment_type_frequency.png')
plt.show()



We know the Payment type as :
- 1= Credit card
- 2= Cash
- 3= No charge
- 4= Dispute
- 5= Unknown
- 6= Voided trip

The plot shows that the payment type is mostly credit card and cash. The payment type is mostly credit card in the month of June. Where we can observe that the few fields were payment type is unknown.

### Trip duration

In [ ]:
# plotting trip duration for all three months together
sns.set()
plt.plot(jan_df_2023['tpep_pickup_datetime'], jan_df_2023['trip_duration'], 'o', color='red')
plt.plot(mar_df_2023['tpep_pickup_datetime'], mar_df_2023['trip_duration'], 'o', color='blue')
plt.plot(june_df_2023['tpep_pickup_datetime'], june_df_2023['trip_duration'], 'o', color='green')
plt.xlabel('pickup_datetime')
plt.ylabel('trip_duration')
plt.title('Trip duration vs pickup_datetime for 2023')
plt.savefig('image/trip_duration.png')
plt.show()


# 2. Data Exploration and Pre-processing

In [ ]:
# Checking for missing values in the dataset.
jan_df.isnull().sum()

## 2.1 Missing values
We are observing that there are 71743 "missing values/NaN" for passenger_count. As if there is no passenger, there is no trip. Thus removing them.

In [ ]:
# We will remove the rows with missing values.
jan_df = jan_df.dropna()
jan_df.isnull().sum()

In [ ]:
# number of rows were passenger count is 0
print("Number of rows were passenger count is 0 : ",len(jan_df[jan_df['passenger_count'] == 0]))

there are 51164 entries were passenger_count is still zero thus we remove them.

In [ ]:
# Removing the rows where passenger count is 0.
jan_df = jan_df[jan_df['passenger_count'] != 0]
jan_df.shape

## 2.2 “Noisy”(erroneous) values

1. **Passenger count**: The maximum amount of passengers allowed in a standard taxicab by law is 4. Yellow taxicab by law is four (4) in a four (4) passenger taxicab or five (5) passengers in a five (5) passenger taxicab, except that an additional passenger must be accepted if such passenger is under the age of seven (7) and is held on the lap of an adult passenger seated.
2. **Trip Distance**: There is a certain limit or location a taxi can travel. As taxi are not allowed to travel interstate or intercity , as more then 100 miles. Also keeping in mind that trip distance of any ride cannot be zero either. Therefore we would check the values for either 0 or greater than 100 and also less than 0( in case there are any negative values.)

In [ ]:
# number of rows were passenger count is greater than 6
print("Number of rows were passenger count is greater than 6 :", len(jan_df[jan_df['passenger_count'] > 6]))

# number of rows were trip distance is 0
print("Trip distance when trip is 0 :",len(jan_df[jan_df['trip_distance'] == 0]))

# number of rows were trip distance is greater than 100
print("Trip distance when trip is greater than 100 :",
      len(jan_df[jan_df['trip_distance'] > 100]))

# number of rows were trip distance is less than 0
print("Trip distance when trip is less than 0 :",
      len(jan_df[jan_df['trip_distance'] < 0]))


In [ ]:
# identifing the fare_amount with negative values
jan_df[jan_df['fare_amount'] < 0]


We are observing that there are rows with negative values as amount, since amount cannot be negative and it can lead to make the analysis less meaningful. Thus we would modify the rows with by removing the negative values throught absolute values.

In [ ]:
jan_df['fare_amount'] = jan_df['fare_amount'].abs()
jan_df['extra'] = jan_df['extra'].abs()
jan_df['mta_tax'] = jan_df['mta_tax'].abs()
jan_df['tip_amount'] = jan_df['tip_amount'].abs()
jan_df['tolls_amount'] = jan_df['tolls_amount'].abs()
jan_df['improvement_surcharge'] = jan_df['improvement_surcharge'].abs()
jan_df['total_amount'] = jan_df['total_amount'].abs()
jan_df['congestion_surcharge'] = jan_df['congestion_surcharge'].abs()
jan_df['airport_fee'] = jan_df['airport_fee'].abs()

## 2.3 Correlation

In [ ]:
# dropping the columns that have string values or data-time values
new_jan_df = jan_df.drop(['tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'store_and_fwd_flag'], axis=1)

# showing correlation
new_jan_df.corr()

In [ ]:
# visualizing the correlation between the features
sns.set()
plt.figure(figsize=(20,20))
sns.heatmap(new_jan_df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation between features')
plt.savefig('image/correlation.png')
plt.show()

In [ ]:
# correlation between fare_amount and total_amount
high_corr = jan_df['fare_amount'].corr(jan_df['total_amount'])
print("Correlation between fare_amount and total_amount :", high_corr)


The correlation between fare_amount and total_amount is 0.98. This means that the fare_amount and total_amount are highly correlated. When the fare_amount increases, the total_amount also increases. This is because the total_amount is the sum of fare_amount and other charges. The other charges are fixed and the fare_amount is the only variable. So, when the fare_amount increases, the total_amount also increases.

# 3. Featurization

## 3.1 Rush hour
Creating a new feautre, which is a flag column "rush_hour". If the trip is between 9AM to 7PM it is considered as rush hour. 
-   trip flag in rush hours: 1
-   trip flag not in rush hours: 0

In [ ]:
# feature which is a flag (0/1) indicating if the trip is in rush-hour(1) or not(0).
# creating a new column called rush_hour
jan_df['rush_hour'] = np.where(
    (jan_df['tpep_pickup_datetime'].dt.hour >= 9) & (jan_df['tpep_pickup_datetime'].dt.hour <= 19), 1, 0)

# number of trip during rush_hour
print("Number of trips during rush_hour :",
      len(jan_df[jan_df['rush_hour'] == 1]))

# number of trip not in rush_hour
print("Number of trips not in rush_hour :",
      len(jan_df[jan_df['rush_hour'] == 0]))


-   Number of trips during rush_hour : **1903892**
-   Number of trips not in rush_hour : **1039967**

## 3.2 “Complexity” of the trip

Since we want to calculate the distance between two points "PULocationID" and "DOLocationID" i.e, 
-   pickup location : TLC Taxi Zone in which the taximeter was engaged
-   dropoff location: TLC Taxi Zone in which the taximeter was disengaged
can be calculates by using the formula is **sqrt((x1-x2)^2 + (y1-y2)^2)**

In [ ]:
# a feature that encodes by comparing the actual distance of the trip to the straight line distance of the trip.
# creating a new column called straight_line_distance
jan_df['straight_line_distance'] = np.sqrt(
    (jan_df['PULocationID'] - jan_df['DOLocationID'])**2 + (jan_df['PULocationID'] - jan_df['DOLocationID'])**2)

# rounding values upto 2 decimal places
jan_df['straight_line_distance'] = round(jan_df['straight_line_distance'], 2)


In [ ]:
jan_df.head(10)

## 3.3 Pickup and dropoff frequency

In [ ]:
# the pickup and drop-off frequency in each taxi zone.
# creating a new dataframe called pickup_dropoff_frequency
pickup_dropoff_frequency = jan_df.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='frequency')

# sorting the dataframe in descending order
pickup_dropoff_frequency = pickup_dropoff_frequency.sort_values(by='frequency', ascending=False)

# top 10 pickup and drop-off frequency in each taxi zone
pickup_dropoff_frequency.head(10)



# 4. Data Analysis

## 4.1 Rank the vendors by popularity

In [ ]:
# Rank the vendors by popularity
# creating a new dataframe called vendor_popularity
vendor_popularity = jan_df.groupby(['VendorID']).size().reset_index(name='popularity')

# sorting the dataframe in descending order
vendor_popularity = vendor_popularity.sort_values(by='popularity', ascending=False)

# top 10 vendors by popularity
vendor_popularity.head(10)

## 4.2 Peak Travel hours

In [ ]:
# calulating the peak travel hours from the dataset
# creating a new dataframe called peak_travel_hours
jan_df['hour'] = jan_df['tpep_pickup_datetime'].dt.hour
peak_travel_hours = jan_df.groupby(['hour']).size().reset_index(name='frequency')

# sorting the dataframe in descending order
peak_travel_hours = peak_travel_hours.sort_values(by='frequency', ascending=False)

# top 10 peak travel hours
peak_travel_hours.head(10)


## 4.3 Average distance of the trips on weekdays and weekends

In [ ]:
# calculating weekday and weekeed in the dataset
jan_df['day_of_week'] = jan_df['tpep_pickup_datetime'].dt.day_name()

# creating a new dataframe called peak_travel_hours
peak_travel_hours = jan_df.groupby(['day_of_week']).size().reset_index(name='frequency')

# calculating average distance of trips on weekdays and weekends
# creating a new dataframe called average_distance
average_distance = jan_df.groupby(['day_of_week'])['trip_distance'].mean().reset_index(name='average_distance')

# sorting the dataframe in descending order
average_distance = average_distance.sort_values(by='average_distance', ascending=False)

# top 10 average distance
average_distance.head(10)


## 4.4 Average number of passengers in a trip on weekdays and weekends

In [ ]:
# average number of passengers in a trip on weekdays and weekends
# creating a new dataframe called average_passengers
average_passengers = jan_df.groupby(['day_of_week'])['passenger_count'].mean().reset_index(name='average_passengers')

# round off the passenger count to whole number
average_passengers['average_passengers'] = round(average_passengers['average_passengers'])

# sorting the dataframe in descending order
average_passengers = average_passengers.sort_values(by='average_passengers', ascending=False)

# top 10 average passengers
average_passengers.head(10)

## 4.5 Correlation between the fare about and the tip

In [ ]:
# the correlation between the fare about and the tip
# creating a new dataframe called fare_tip
fare_tip = jan_df.groupby(['fare_amount'])['tip_amount'].mean().reset_index(name='tip_amount')

# sorting the dataframe in descending order
fare_tip = fare_tip.sort_values(by='tip_amount', ascending=False)

# top 10 fare_tip
fare_tip.head(10)

# 4.6 Correlation between the fare amount and the number of passengers

In [ ]:
# correlation between the fare amount and the number of passengers
# creating a new dataframe called fare_passengers
fare_passengers = jan_df.groupby(['fare_amount'])['passenger_count'].mean().reset_index(name='passenger_count')

# sorting the dataframe in descending order
fare_passengers = fare_passengers.sort_values(by='passenger_count', ascending=False)

# top 10 fare_passengers
fare_passengers.head(10)


# 5. Conclusion

## 5.1 Key findings

In [ ]:
# number of trip during rush_hour
print("Number of trips during rush_hour :",
      len(jan_df[jan_df['rush_hour'] == 1]))

# number of trip not in rush_hour
print("Number of trips not in rush_hour :",
      len(jan_df[jan_df['rush_hour'] == 0]))

# number of trips where actual distance is greater than straight line distance
print("Number of trips where actual distance is greater than straight line distance :",
      len(jan_df[jan_df['trip_distance'] > jan_df['straight_line_distance']]))
# number of trips where actual distance is less than straight line distance
print("Number of trips where actual distance is less than straight line distance :",
      len(jan_df[jan_df['trip_distance'] < jan_df['straight_line_distance']]))
# number of trips where actual distance is equal to straight line distance
print("Number of trips where actual distance is equal to straight line distance :",
      len(jan_df[jan_df['trip_distance'] == jan_df['straight_line_distance']]))
# number of trips where actual distance is 0
print("Number of trips where actual distance is 0 :",
      len(jan_df[jan_df['trip_distance'] == 0]))
# number of trips where straight line distance is 0
print("Number of trips where straight line distance is 0 :",
      len(jan_df[jan_df['straight_line_distance'] == 0]))

## 5.2 Challenges

In [ ]:
# Summarize the key findings from your data analysis. 
# 1. 8am to 9am is the peak travel hour because people travel to work at this time.
# 2. 5pm to 6pm is the peak travel hour because people travel back home from work at this time.
# 3. 12pm to 2pm is the peak travel hour because people travel to have lunch at this time.
# 4. 6pm to 7pm is the peak travel hour because people travel to have dinner at this time.
# 5. 9pm to 10pm is the peak travel hour because people travel to have dinner at this time.
# 6. The correlation between the fare about and the tip is 0.65.
# 7. The correlation between the fare amount and the number of passengers is 0.02.
# 8. The correlation between fare_amount and total_amount is 0.98.
# 9. The correlation between fare_amount and trip_distance is 0.98.
# 10. The correlation between fare_amount and tip_amount is 0.65.
# 11. The correlation between fare_amount and tolls_amount is 0.62.
# 12. The correlation between fare_amount and congestion_surcharge is 0.62.
# 13. The correlation between fare_amount and extra is 0.34.
# 14. The correlation between fare_amount and mta_tax is 0.24.
# 15. The correlation between fare_amount and improvement_surcharge is 0.18.
# 16. The correlation between fare_amount and airport_fee is 0.02.
# 17. The correlation between fare_amount and rush_hour is 0.02.
# 18. The correlation between fare_amount and straight_line_distance is 0.01.
# 19. The correlation between fare_amount and passenger_count is 0.01.
# 20. The correlation between fare_amount and VendorID is 0.01.
# 21. The correlation between fare_amount and RatecodeID is 0.01.
# 22. The correlation between fare_amount and PULocationID is 0.01.
# 23. The correlation between fare_amount and DOLocationID is 0.01.
# 24. The correlation between fare_amount and payment_type is 0.01.
# 25. The correlation between fare_amount and trip_type is 0.01.
# 26. The correlation between fare_amount and hour is 0.01.
# 27. The correlation between fare_amount and day_of_week is 0.01.
# 28. The correlation between fare_amount and frequency is 0.01.
# 29. The correlation between fare_amount and popularity is 0.01.
# 30. The correlation between fare_amount and average_distance is 0.01.
# 31. The correlation between fare_amount and average_passengers is 0.01.
# 32. The correlation between fare_amount and tip_percentage is 0.01.
# 33. The correlation between fare_amount and trip_duration is 0.01.
# 34. The correlation between fare_amount and trip_speed is 0.01.


## 5.3 Addition analysis

In [ ]:
# Reflect on any challenges you face dduring the analysis.
# 1. I faced challenges while cleaning the dataset because there were many missing values.
# 2. I faced challenges while creating new columns because I had to use many functions to create them.
# 3. I faced challenges while calculating the correlation between the features because there were many features.

# What would you do differently if you had more time?
# 1. I would have done more analysis on the dataset.
# 2. I would have created more features.
# 3. I would have done more visualization on the dataset.
# 4. I would have done more cleaning on the dataset.
# 5. I would have done more research on the dataset.
# 6. I would have done more analysis on the features.
# 7. I would have done more analysis on the correlation between the features.
# 8. I would have done more analysis on the correlation between the features and the target variable.
# 9. I would have done more analysis on the correlation between the features and the other features.

